# Target variables notebook
This notebook describes the functions that compute the target variables for the sampled windows in the notebook `4-ak-window-sampling`. In addition, this notebook is responsible for computing various other statistics that are relevant (necessary) for the computation of the target variables, like the mean SCL during the baseline component. These are stored in the `data\information` directory. 

#### Requirements
If one wants to run this notebook, make sure that you have run the `1-ak-physio-dataframe` notebook. This notebook is responsible for the creation of the physiological DataFrames, stored in `data\interim`, in the 3 folders `physiological`, `physiological_baseline` and `physiological_all`.

In [1]:
import pandas as pd
import numpy as np
import os
import csv
import neurokit2 as nk

In [2]:
Mitchel = False

After loading in the required modules, we store the working directory in a variable called `project_dir`. We then store the folder where the data files are located in a variable called `data_dir`. We also store the different paths of the directories that hold the physiological DataFrames in variables, aswell as the list of the files in these directories.

In [3]:
project_dir = os.getcwd().split('\\')[:-1] 
project_dir = '\\'.join(project_dir) # Get the project dir

# Get the data dir
if Mitchel:
    data_dir = 'C:\\Users\\mitch\\OneDrive - UGent\\UGent\\Projects\\7. tDCS_Stress_WM_deSmet\\data'
    data_dir = 'Z:\\ghep_lab\\2020_DeSmetKappen_tDCS_Stress_WM_VIDEO\\Data'
else: 
    data_dir = project_dir + '\\data'
# Get the specific physiological directories and the files in these dirs and store these in respectively named variables
physio_dir = data_dir+'\\interim\\physiological'
physio_files = [file for file in os.listdir(physio_dir)]
baseline_dir = data_dir + '\\interim\\physiological_baseline'
baseline_files = [file for file in os.listdir(baseline_dir)]
all_dir = data_dir + '\\interim\\physiological_all'
all_files = [file for file in os.listdir(all_dir)]

### Compute baseline and ALL components statistics
The cells below are responsible for the computation of the various statistics that are used for computing the target variables during the window sampling process. 

#### EDA Statistics
The following cells concern the SCL, or Tonic EDA, signal. The first cell contains the function that can calculate the various statistics for the SCL signal. It calculates the mean and std for an arbitrary period of raw EDA signal, aswell as the min and maximum. 

In the next cell this function is then applied to both the baseline and all DataFrames, to calculate these descriptives statistics. This information is then stored in dataframes, which are stored `data\information\SCL stats`. The function used in the window processing to calculate the target variables, uses these DataFrames in its computation. Therefore, the cells below need to be run before one can execute the `4-ak-window-sampling` notebook. 

In [4]:
def compute_PP_EDA(physio_data: pd.DataFrame, pp: int) -> dict:
    """This function computes the mean, std, min and max SCL of a given raw EDA signal. It returns these stats and the pp id in a dict"""
    
    processed = {} # Create the dict that will store the stats and is returned at the end of the function
    
    freq=round(1/(physio_data.timestamp.values[1] - physio_data.timestamp.values[0])) # Calculate the frequency at which the raw EDA signal was sampled 
    
    signals, info = nk.eda_process(physio_data.raw_EDA.dropna(), sampling_rate=freq) # Process the EDA signals using the eda_process function from NeuroKit2
    
    # Compute the various SCL statistics and store this in a dict, before returning this dict 
    processed['mean_SCL'] = signals.EDA_Tonic.mean()
    processed['std_SCL'] = signals.EDA_Tonic.std()
    processed['max_SCL'] = signals.EDA_Tonic.max()
    processed['min_SCL'] = signals.EDA_Tonic.min()
    processed['pp'] = pp
    return processed

In [5]:
## Compute the EDA stat for the baseline component
df = [] # Create an empty list to store the dicts that will be returned by the compute_PP_EDA function

for file in baseline_files: # For each baseline file
    data = pd.read_feather(baseline_dir + '\\' + file) # Read in the DataFrame
    pp = int(file[:-8]) # Get the pp id
    df.append(compute_PP_EDA(data, pp)) # Compute the stats and add this to the empty list

PP_EDA = pd.DataFrame(df) # Convert this list of dicts into a dataframe

## Store the different stats in this dataframe in a csv file each
## This way they can be accessed independently and only when necessary
pd.DataFrame({'mean_SCL':PP_EDA.mean_SCL.values}, index=PP_EDA.pp).to_csv(data_dir +  '\\information\\SCL stats\\PP_meanSCL_Baseline.csv')
pd.DataFrame({'std_SCL':PP_EDA.std_SCL.values}, index=PP_EDA.pp).to_csv(data_dir +  '\\information\\SCL stats\\PP_stdSCL_Baseline.csv')
pd.DataFrame({'min_SCL':PP_EDA.min_SCL.values}, index=PP_EDA.pp).to_csv(data_dir +  '\\information\\SCL stats\\PP_minSCL_Baseline.csv')
pd.DataFrame({'max_SCL':PP_EDA.max_SCL.values}, index=PP_EDA.pp).to_csv(data_dir +  '\\information\\SCL stats\\PP_maxSCL_Baseline.csv')


## Compute the EDA stat for all the active components
df = [] # Create an empty list to store the dicts that will be returned by the compute_PP_EDA function
for file in all_files: # For each all file
    data = pd.read_feather(all_dir + '\\' + file) # Read in the DataFrame
    pp = int(file[:-8]) # Get the pp id
    df.append(compute_PP_EDA(data, pp)) # Compute the stats and add this to the empty list

PP_EDA = pd.DataFrame(df) # Convert this list of dicts into a dataframe

## Store the different stats in this dataframe in a csv file each (by first creating a separate DataFrame for a stat and the using the to_csv method)
## This way they can be accessed independently and only when necessary 
pd.DataFrame({'mean_SCL':PP_EDA.mean_SCL.values}, index=PP_EDA.pp).to_csv(data_dir +  '\\information\\SCL stats\\PP_meanSCL_all.csv')
pd.DataFrame({'std_SCL':PP_EDA.std_SCL.values}, index=PP_EDA.pp).to_csv(data_dir +  '\\information\\SCL stats\\PP_stdSCL_all.csv')
pd.DataFrame({'min_SCL':PP_EDA.min_SCL.values}, index=PP_EDA.pp).to_csv(data_dir +  '\\information\\SCL stats\\PP_minSCL_all.csv')
pd.DataFrame({'max_SCL':PP_EDA.max_SCL.values}, index=PP_EDA.pp).to_csv(data_dir +  '\\information\\SCL stats\\PP_maxSCL_all.csv')

#### HRV Statistics
The following cells concern the ECG signal, which is used to compute various HRV signals.The first cell contains the function that can calculate the various HRV measures for an arbitrary raw ECG signal. This function computes the meanNN, RMSSD and SDNN HRV measures. 

In the next cell this function is then applied to both the baseline and all DataFrames, to calculate these HRV measures. This information is then stored in dataframes, which are stored `data\information\HRV stats`. The function used in the window processing to compute the target variables, uses these DataFrames in its computation. Therefore, the cells below also need to be run before one can execute the `4-ak-window-sampling` notebook. 

In [6]:
def compute_PP_HRV(physio_data: pd.DataFrame, pp: int) -> dict:
    """This function computes the meanNN, RMSDD and SDNN HRV of a given raw ECG signal. It returns these measures and the pp id in a dict"""
    processed = {} # Create the dict that will store the measures and is returned at the end of the function
    
    freq=round(1/(physio_data.timestamp.values[1] - physio_data.timestamp.values[0])) # Calculate the frequency at which the raw ECG signal was sampled 
    
    ecg_signals, info = nk.ecg_process(data["raw_ECG"], sampling_rate=freq) # Get the clean ECG signal using the ecg_process function from NeuroKit2
    ecg_features = nk.ecg_intervalrelated(ecg_signals) # Compute HRV measures using the ecg_intervalrelated function from NeuroKit2
    
    # Compute the various SCL statistics and store this in a dict, before returning this dict 
    processed['HRV_MeanNN'] = ecg_features['HRV_MeanNN'].values[0]
    processed['HRV_RMSSD'] = ecg_features['HRV_RMSSD'].values[0]
    processed['HRV_SDNN'] = ecg_features['HRV_SDNN'].values[0]
    processed['pp'] = pp
    return processed

In [ ]:
## Compute the HRV measures for the baseline component
df = [] # Create an empty list to store the dicts that will be returned by the compute_PP_HRV function
for file in baseline_files: # For each baseline file
    data = pd.read_feather(baseline_dir + '\\' + file) # Read in the DataFrame
    pp = int(file[:-8]) # Get the pp id
    df.append(compute_PP_HRV(data, pp)) # Compute the HRV measures and add the returned dict to the list
    
PP_ECG = pd.DataFrame(df)# Convert this list of dicts into a dataframe

## Store the different stats in this dataframe in a csv file each (by first creating a separate DataFrame for a stat and the using the to_csv method)
## This way they can be accessed independently and only when necessary
pd.DataFrame({'HRV_RMSSD':PP_ECG.HRV_RMSSD.values}, index=PP_ECG.pp).to_csv(data_dir +  '\\information\\HRV stats\\PP_RMSSD_Baseline.csv')
pd.DataFrame({'HRV_MeanNN':PP_ECG.HRV_MeanNN.values}, index=PP_ECG.pp).to_csv(data_dir +  '\\information\\HRV stats\\PP_MeanNN_Baseline.csv')
pd.DataFrame({'HRV_SDNN':PP_ECG.HRV_SDNN.values}, index=PP_ECG.pp).to_csv(data_dir +  '\\information\\HRV stats\\PP_SDNN_Baseline.csv')

### Target variable functions
The two cells below contain the functions that are used in the `4-ak-window-sampling` notebook. The functions are also stored in the `targetComputation.py` script from which they are imported. Both functions are very similar to their counterpart that computed the SCL stats and HRV measures in the previous cells. They differ in that they use the computed information to calculate the target variables.

In [ ]:
def compute_EDA_Target(physio_data: pd.DataFrame) -> dict:
    """Computes the EDA Target variables from an abritray dataframe containing the raw EDA signal. Returns these target variables in a dict."""
    
    pp = physio_data.pp.values[0] # Get the pp id
    
    ## Open the baseline, and all descriptive files, then get the value that corresponds to the current pp id and store these in corresponding variables
    ## Baseline descriptives
    mean_Baseline = pd.read_csv(data_dir +  '\\information\\SCL stats\\PP_meanSCL_Baseline.csv', index_col=0)['mean_SCL'].to_dict()[pp]
    min_Baseline = pd.read_csv(data_dir +  '\\information\\SCL stats\\PP_minSCL_Baseline.csv', index_col=0)['min_SCL'].to_dict()[pp]
    ## All Descriptives
    mean_all = pd.read_csv(data_dir +  '\\information\\SCL stats\\PP_meanSCL_all.csv', index_col=0)['mean_SCL'].to_dict()[pp]
    std_all = pd.read_csv(data_dir +  '\\information\\SCL stats\\PP_stdSCL_all.csv', index_col=0)['std_SCL'].to_dict()[pp]
    max_all = pd.read_csv(data_dir +  '\\information\\SCL stats\\PP_maxSCL_all.csv', index_col=0)['max_SCL'].to_dict()[pp]
    
    processed = {} # Create the dict to store the target variables in and which is returned later in the function
    
    df = physio_data.dropna() # Drop the empty rows in the function (this occurs due to differences in sampling frequency between ECG and EDA signal in the AMSDATA files)
    
    freq=round(1/(df.t_from_start.values[1] - df.t_from_start.values[0])) # Calculate the frequency at which the raw EDA signal was sampled 
    seconds = df.t_from_start.values[-1] - df.t_from_start.values[0] # Calculate the amount seconds the physio signal contains
    
    signals, info = nk.eda_process(data.raw_EDA.dropna(), sampling_rate=freq) # Process the EDA signals using the eda_process function from NeuroKit2
    
    ## Compute various EDA target variables and store these in the dict. Also add the pp id to the dict and return this dict
    processed['mean_SCL'] = signals.EDA_Tonic.mean() # Compute the mean SCL signal
    processed['corrected_mean_SCL'] = processed['mean_SCL'] - mean_Baseline # Compute the corrected mean SCL, by subtracting the mean SCL from the baseline component
    processed['range_corrected_mean_SCL'] = ((signals.EDA_Tonic - min_Baseline) / (max_SCL - min_Baseline)).mean() # Compute the range corected min SCL, by Min Max scaling the signal and before computing the mean
    processed['standardised_mean_scl'] = ((signals.EDA_Tonic - mean_all) / std_all).mean() # Compute the standarised mean SCL, by standardising using the mean and std SCL throughout all the active components
    processed['frequency_NS_SCR'] = len(info['SCR_Peaks'])/seconds * 60 # Compute the frequency in which Non-Stimulus Skin Conductance Responses (NS-SCRs) occured, which is a EDA measures related to the Phasic component
    return processed

In [ ]:
def compute_ECG_Targets(physio_data: pd.DataFrame) -> dict:
    """Computes the EDA Target variables from an abritray dataframe containing the raw EDA signal. Returns these target variables in a dict."""
    
    pp = physio_data.pp.values[0] # Get the pp id
    
    # Open the baseline HRV measures DataFrames, and then get the value that corresponds to the current pp id and store these in corresponding variables
    HRV_RMSSD_baseline = pd.read_csv(data_dir +  '\\information\\HRV stats\\PP_RMSSD_Baseline.csv', index_col=0)['HRV_RMSSD'].to_dict()[pp]
    HRV_MeanNN_baseline = pd.read_csv(data_dir +  '\\information\\HRV stats\\PP_MeanNN_Baseline.csv', index_col=0)['HRV_MeanNN'].to_dict()[pp]
    HRV_SDNN_baseline = pd.read_csv(data_dir +  '\\information\\HRV stats\\PP_SDNN_Baseline.csv', index_col=0)['HRV_SDNN'].to_dict()[pp]    
    
    processed = {} # Create the dict to store the target variables in and which is returned later in the function
    
    freq=round(1/(physio_data.t_from_start.values[1] - physio_data.t_from_start.values[0])) # Calculate the frequency at which the raw EDA signal was sampled 
    seconds = physio_data.t_from_start.values[-1] - physio_data.t_from_start.values[0] # Calculate the amount seconds the physio signal contains
    
    ecg_signals, info = nk.ecg_process(data["raw_ECG"], sampling_rate=freq) # Get the clean ECG signal using the ecg_process function from neurokit2    
    ecg_features = nk.ecg_intervalrelated(ecg_signals) # Compute HRV measures using the ecg_intervalrelated function from NeuroKit2
    
    ## Compute various HRV target variables and store these in the dict. Also add the pp id to the dict and return this dict
    processed['HRV_MeanNN'] = ecg_features['HRV_MeanNN'].values[0] # Store the mean of the time interval between the NN peaks
    processed['HRV_RMSSD'] = ecg_features['HRV_RMSSD'].values[0] # Store the Root Mean Squared Standard deviation of the time interval between the NN peaks 
    processed['HRV_SDNN'] = ecg_features['HRV_SDNN'].values[0] # Store the standard deviation of the time interval between the NN peaks
    processed['HRV_MeanNN_corrected'] = processed['HRV_MeanNN'] - HRV_MeanNN_baseline # Correct the meanNN by subtracting with the meanNN from the baseline component and store this 
    processed['HRV_RMSSD_corrected'] = processed['HRV_RMSSD'] - HRV_RMSSD_baseline # Correct the RMSSD by subtracting with the RMSSD from the baseline component and store this
    processed['HRV_SDNN_corrected'] = processed['HRV_SDNN'] - HRV_SDNN_baseline # Correct the SDNN by subtracting with the SDNN from the baseline component and store this
    return processed